# 数値解析2020 レポート

**VII-B.** 
(1) 3次の自然スプライン補間を用いて3つのデータ点 $(x_0,y_0)=(0,1)$, $(x_1,y_1)=(1,2)$, $(x_2,y_2)=(4,2)$ を通る2本の3次の区分的補間多項式 $P_i(x) = y_i + a_i (x-x_i) + b_i (x-x_i)^2 + c_i (x-x_i)^3$, $(i=0,1)$ を求めよ。

(2) 3次の自然スプライン補間を用いて5つのデータ点 $(x_0,y_0)=(0,1)$, $(x_1,y_1)=(1,2)$, $(x_2,y_2)=(4,2)$, $(x_3,y_3)=(5,1)$, $(x_4,y_4)=(6,0)$ を通る4本の3次の区分的補間多項式 $P_i(x) = y_i + a_i (x-x_i) + b_i (x-x_i)^2 + c_i (x-x_i)^3$, $(i=0,1,2,3)$ を求めるプログラムを作成し、$a_i, b_i, c_i$ $(i=0,1,2,3)$ を有効数字10進3桁で4桁目を四捨五入して答えよ。

以下Python3を用いたプログラム例を示す。

In [1]:
# Pythonのバージョン確認
import sys
print(sys.version)

3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]


# レポート解答用冗長コード

In [2]:
import numpy as np
n = 4
x = np.array([0, 1, 4, 5, 6])
y = np.array([1, 2, 2, 1, 0])

# Preparation

h = np.zeros(n)
dy = np.zeros(n)

for i in range(n):
    h[i] = x[i+1] - x[i]
    dy[i] = y[i+1] - y[i]

# Constraction of tridiagnal matrix

d = np.array(np.zeros(n))
for i in range(n):
    d[i] = 1

u = np.array(np.zeros(n)) # u[0] will not be used
u[1] = 0
for i in range(2, n):
    u[i] = (1./2) * h[i-1]/(h[i-1] + h[i-2])

l = np.array(np.zeros(n-1))
l[0] = 0
for i in range(1,n-1):
    l[i] = (1./2) * h[i]/(h[i+1] + h[i])

g = np.array(np.zeros(n))
g[0] = 0
for i in range(1, n):
    g[i] = (3./2) * (dy[i]/h[i] - dy[i-1]/h[i-1]) / (h[i]+h[i-1])

# Solve tridiagonal systems

b = np.zeros(n)

for i in range(1, n):
    d[i] = d[i] - u[i] * l[i-1] / d[i-1]
    g[i] = g[i] - g[i-1] * l[i-1] / d[i-1]

b[n-1] = g[n-1] / d[n-1]

for i in reversed(range(1,n-1)):
    b[i] = (g[i] - u[i+1] * b[i+1])/d[i]

# Calculate c[i] and a[i] from b[i]
a = np.zeros(n)
c = np.zeros(n)
for i in range(n-1):
    a[i] = dy[i]/h[i] - (1./3) * (2*b[i] + b[i+1]) * h[i]
    c[i] = (1./3) * (b[i+1]-b[i])/ h[i]
a[n-1] = dy[n-1]/h[n-1] - (2./3) * b[n-1] * h[n-1]
c[n-1] =  - (1./3) * b[n-1] / h[n-1]

print()
print("a", a)
print("b", b)
print("c", c)
for i in range(n):
    #print(a[i], b[i], c[i])
    print("p_{5:d} (x) = {1:e} + {2:e} * (x - {0:e})  + {3:e} * (x - {0:e})**2 + {4:e} * (x - {0:e})**3".format(x[i], y[i], a[i], b[i], c[i], i))


a [ 1.08962264  0.82075472 -0.83490566 -1.04716981]
b [ 0.         -0.26886792 -0.28301887  0.07075472]
c [-0.08962264 -0.00157233  0.11792453 -0.02358491]
p_0 (x) = 1.000000e+00 + 1.089623e+00 * (x - 0.000000e+00)  + 0.000000e+00 * (x - 0.000000e+00)**2 + -8.962264e-02 * (x - 0.000000e+00)**3
p_1 (x) = 2.000000e+00 + 8.207547e-01 * (x - 1.000000e+00)  + -2.688679e-01 * (x - 1.000000e+00)**2 + -1.572327e-03 * (x - 1.000000e+00)**3
p_2 (x) = 2.000000e+00 + -8.349057e-01 * (x - 4.000000e+00)  + -2.830189e-01 * (x - 4.000000e+00)**2 + 1.179245e-01 * (x - 4.000000e+00)**3
p_3 (x) = 1.000000e+00 + -1.047170e+00 * (x - 5.000000e+00)  + 7.075472e-02 * (x - 5.000000e+00)**2 + -2.358491e-02 * (x - 5.000000e+00)**3


# 簡略版

In [3]:
import numpy as np
n = 4
#n = 2
x = np.array([0, 1, 4, 5, 6])
#x = np.array([0, 1, 4])
y = np.array([1, 2, 2, 1, 0])
#y = np.array([1, 2, 2])

h = np.array([x[i+1] - x[i] for i in range(n)])
dy = np.array([y[i+1] - y[i] for i in range(n)])


d = np.array(np.ones(n))
u = np.array([0, 0] + [ h[i-1]/(2*(h[i-1] + h[i-2]) ) for i in range(2, n) ])
l = np.array([0] + [ h[i]/(2*(h[i+1] + h[i])) for i in range(1, n-1) ])
g = 3./2 * np.array([0] + [ (dy[i]/h[i] - dy[i-1]/h[i-1])/(h[i]+h[i-1]) for i in range(1, n)])

for i in range(1, n):
    d[i] = d[i] - u[i] * l[i-1] / d[i-1]
    g[i] = g[i] - g[i-1] * l[i-1] / d[i-1]

b = np.zeros(n)

b[n-1] = g[n-1] / d[n-1]

for i in reversed(range(1, n-1)):
    b[i] = (g[i] - u[i+1] * b[i+1])/d[i]

a = np.array([dy[i]/h[i] - 1/3 * (2*b[i] + b[i+1]) * h[i] for i in range(n-1)] + [dy[n-1]/h[n-1] - 2/3 * b[n-1] * h[n-1]])
c = np.array( [ (b[i+1] - b[i])/(3 * h[i]) for i in range(n-1)] + [ - b[n-1] / (3 * h[n-1])])

print(a)
print(b)
print(c)

for i in range(n):
    #print(a[i], b[i], c[i])
    print("p_{5:d} (x) = {1:e} + {2:e} * (x - {0:e})  + {3:e} * (x - {0:e})**2 + {4:e} * (x - {0:e})**3".format(x[i], y[i], a[i], b[i], c[i], i))

[ 1.08962264  0.82075472 -0.83490566 -1.04716981]
[ 0.         -0.26886792 -0.28301887  0.07075472]
[-0.08962264 -0.00157233  0.11792453 -0.02358491]
p_0 (x) = 1.000000e+00 + 1.089623e+00 * (x - 0.000000e+00)  + 0.000000e+00 * (x - 0.000000e+00)**2 + -8.962264e-02 * (x - 0.000000e+00)**3
p_1 (x) = 2.000000e+00 + 8.207547e-01 * (x - 1.000000e+00)  + -2.688679e-01 * (x - 1.000000e+00)**2 + -1.572327e-03 * (x - 1.000000e+00)**3
p_2 (x) = 2.000000e+00 + -8.349057e-01 * (x - 4.000000e+00)  + -2.830189e-01 * (x - 4.000000e+00)**2 + 1.179245e-01 * (x - 4.000000e+00)**3
p_3 (x) = 1.000000e+00 + -1.047170e+00 * (x - 5.000000e+00)  + 7.075472e-02 * (x - 5.000000e+00)**2 + -2.358491e-02 * (x - 5.000000e+00)**3
